In [5]:
#node for code tests

import importlib 
# importlib.reload(initialization)
from utils import *

# list_available_com_ports()
hap = initialize_hap()
client,addr = initialize_al()

# rough calibration
al_2_robot_tf = get_al_2_robot_tf()

socket created 
socket binded to port 
socket listensing ... 
accepted


In [ ]:
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
from IPython.display import clear_output
# from _thread import *
import threading
import csv
from csv import writer

tag = []
state = []
device_position_r = []
velocity = []
lock = threading.Lock()

def process_data_from_AL():
    global tag,state,device_position_r,velocity,lock
    print("thread has started")
    while True :         
        incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
        data_start_ix = incoming_message.find("[")
        data_end_ix = incoming_message.find("]")
        data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)

        while(not data_found):
            incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
            data_start_ix = incoming_message.find("[")
            data_end_ix = incoming_message.find("]")
            data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)
        
        incoming_message = incoming_message[data_start_ix+1 : data_end_ix] 
        incoming_message = incoming_message.replace(',','.')
        list_matrix = incoming_message.split(";")
        lock.acquire()
        state,position,velocity = (list_matrix[1],
                                     np.fromstring(list_matrix[2], dtype=np.float,sep =" "),
                                     np.fromstring(list_matrix[3], dtype=np.float,sep =" "))
        #swap_z 
#         print("velocity norm", np.linalg.norm(velocity))
        position[2] *= -1 
        position_al = np.hstack((position,1))
        device_position_r = tf2robot_new @ position_al
#         print("device_position_r",device_position_r)
#         clear_output(wait=True)
        lock.release()

def check_distance_ee_al():
    time.sleep(2)
    triggered = False
    start_time = time.perf_counter_ns() 
    while True:
        distance = np.linalg.norm(np.array(device_position_r[:3]) - np.array(rob.getl())[:3])
        velocity_magnitude = np.linalg.norm(velocity)
        elapsed_time = (time.perf_counter_ns() - start_time)/ 1e9 
        write_data(distance, velocity_magnitude, device_position_r,elapsed_time)   
        if(not triggered and distance < 0.35):
            start_time = time.perf_counter_ns()
            hap_thread.start()
            print("reached active distance", distance)
            triggered = True
#         if(velocity_magnitude > 0.1 and not triggered):
# #             print("user has made a terrible mistake")
#             write_data(distance, velocity_magnitude, device_position_r,elapsed_time)  
        if(velocity_magnitude > 0.1 and triggered):
            reaction_time = (time.perf_counter_ns() - start_time)/ 1e9
            write_data(distance, velocity_magnitude, device_position_r,elapsed_time)  
            print("out of active distance", distance,reaction_time)


def write_data(writer_object,data):
    writer_object.writerow(data)

data_from_al_thread = threading.Thread(target = process_data_from_AL, daemon = True, args=())
data_from_al_thread.start()

reaction_time = 0.1

check_distance_thread = threading.Thread(target = check_distance_ee_al, daemon = True, args=())
check_distance_thread.start()

hap_thread = threading.Thread(target = hap_action, daemon = True, args=(hap))

a = 0.05
v = 0.05
    
initial_robot_pose = rob.getl()
pose = initial_robot_pose.copy()
triggered = False   

try:
    with open('all_data.cvs',  mode='a') as all_data, open('events.csv', 'a') as events:
        writer_all_data = writer(all_data)
        writer_events = writer(events)
        
        while True:
            print("robot tcp is at: ", rob.getl())
            print("move the hand to a new position and press esc")
            keyboard.wait('esc')

            direction = device_position_r[:3] - initial_robot_pose[:3]
            dir_norm = direction / np.linalg.norm(direction)

            pose[:3] = device_position_r[:3]  - dir_norm * 0.15 
            print("press to move closer to tracker ",pose)

            keyboard.wait('esc')
            rob.movel(pose, acc = a, vel = v)
            rob.movel(initial_robot_pose, acc = a, vel = v)
            clear_output(wait=True)
            triggered = False
except:
        f_object.close()
        c.shutdown(socket.SHUT_RDWR)
        s.close()
        sys.exit()        

In [ ]:
#nodes for data collection
from scipy.spatial.transform import Rotation as R
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
# from IPython.display import clear_output
from _thread import *
import threading

#### HAP
try:
    hap = serial.Serial("COM4", baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial("COM4", baudrate=115200, timeout=0.1)
    hap.close()
    
    print("No connected to hap")


#hap.write(bytes(str(35) + "," + str(10)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "\n",'utf-8'))
# time.sleep(1.0)
 i= 15
while i < 60:

    hap.write(bytes(str(i) + "," + str(10)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "\n",'utf-8'))
    i +=1
    print(i)
    time.sleep(2.0)
hap.close()
                              

# with open('data.csv',  mode='a') as f_object:
#         writer_object = writer(f_object)
#         row = np.append(np.concatenate((x,y,z)), time.time())
#         writer_object.writerow(row)

#         f_object.close()

In [ ]:
hap.close()

In [ ]:
# Custom codein the mix  

import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
# from _thread import *
import threading
import time

def get_pose():
    global rob, go, time1
    i = 0
    while go:
        i+=1
        prox = rob.getl()
#         if((time.perf_counter_ns()- time1)/1e9 > 5):
#             rob.stopl()
#         print(prox)
    print(i)
    
# from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper

# # rob.set_tcp((0,0,0,0,0,0))
# #     rob.set_payload(0.5, (0,0,0))
go = True
p = threading.Thread(target = get_pose, args = (),daemon = True)
p.start()

l = 0.25
v = 0.25
a = 0.25
time1 = time.perf_counter_ns() 
# pose = rob.getl()
# print("robot tcp is at: ", pose)
# print("absolute move in base coordinate ")
# pose[2] += l
# rob.movel(pose, acc=a, vel=v)
# print("relative move in base coordinate ")
rob.translate((l, 0, 0), acc = a, vel = v)
rob.translate((-l, 0, 0), acc = a, vel = v)

# print("relative move back and forth in tool coordinate")

# rob.translate_tool((-l, 0, 0), acc=a, vel=v)
# rob.translate_tool((l, 0, 0), acc=a, vel=v)

print((time.perf_counter_ns() - time1)/ 1e9 )
go = False
#     rob.close()

In [ ]:
import numpy as np
p_tool_r = np.array([0.612870852821809, 0.4789758999604802, 0.5050407343318414,1])
pose = rob.getl() 
pose[:3] = p_tool_r[:3]
rob.movel(pose, acc = 0.05, vel = 0.05)

In [ ]:
hap_do(hap)

In [ ]:
hap.close()

In [ ]:
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output

tag = []
state = []
device_position_r = []
velocity = []

def process_data_from_AL():
    global tag,state,device_position_r,velocity,lock
    print("thread has started")
    while True :         
        incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
        data_start_ix = incoming_message.find("[")
        data_end_ix = incoming_message.find("]")
        data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)

        while(not data_found):
            incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
            data_start_ix = incoming_message.find("[")
            data_end_ix = incoming_message.find("]")
            data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)
        
        incoming_message = incoming_message[data_start_ix+1 : data_end_ix] 
        incoming_message = incoming_message.replace(',','.')
        list_matrix = incoming_message.split(";")
        state,position,velocity = (list_matrix[1],
                                     np.fromstring(list_matrix[2], dtype=np.float,sep =" "),
                                     np.fromstring(list_matrix[3], dtype=np.float,sep =" "))
        #swap_z 

        position[2] *= -1 
        position_al = np.hstack((position,1))
        device_position_r = tf2robot_new @ position_al
        print("position_al",position_al)
        print("device_position_r",device_position_r)
        clear_output(wait=True)

s = socket.socket()

print('socket created ')
port = 20
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
while True:
    process_data_from_AL()

In [ ]:
import urx

try:
    rob = urx.Robot("192.168.88.162")
except:
    print('robot error')
    interval = False
    
import time, serial

def hap_do(s):
    x1=20
    y1=-14
    v1 = 0
    v2 = 0
    x3= 100
    y3= 0
    c = 0

    while (c < 30):
        c += 2
        s.write(bytes(str(x1) + "," + str(y1)+ "," + str(v1)+ "," + str(v2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
        time.sleep(0.1)
        x1 += 2


    x1=35
    y1=10
    x2=0
    y2=0
    x3= 0
    y3= 0

    s.write(bytes(str(x1) + "," + str(y1)+ "," + str(x2)+ "," + str(y2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

# try:
#     hap = serial.Serial('COM3', baudrate=115200, timeout=0.1)
#     print("connected")
# except serial.SerialException:
#     print("not connected")
#     hap = serial.Serial('COM3', baudrate=115200, timeout=0.1)
#     hap.close()

hap_do(hap)

In [ ]:
hap_do(hap)

In [ ]:
hap_do(hap)

In [ ]:
import csv
from csv import writer

# f = open("test.cvs","w+")
x = 0
y = 1
z = 2
time = 3
# f.close()
with open('test.cvs',  mode='a') as f_object:
    row = [x,y,z,time]
    writer_object = writer(f_object)
    writer_object.writerow(row)
    print("yahoo ую")

In [ ]:
import numpy as np

#It is assumed that axes are oriented correctly
# shitty calibration values
#the base is slightly lower
# shitty position of a robot base in AL. z is swapped 
p_al = np.array([-1.13,1.10,0.41, 1])

tf2robot = np.array([[0,0,1,-p_al[2]],[1,0,0,-p_al[0]],[0,1,0,-p_al[1]],[0,0,0,1]],dtype = np.float32)
print(tf2robot)
# print(tf2robot tf_from_al_to_tool_al)
# l = "0.0 1.7 0.0"
# a = np.fromstring (l, dtype=np.float,sep =" ")
# b = [a,a,l]
# b

In [ ]:
import sys
import keyboard

keyboard.add_hotkey('alt', lambda : print('foobar'))
# keyboard.hook_key('alt', lambda e: print('foobar'))
try:
    while True:
        pass
except:
    pass
finally:
    keyboard.unhook_all_hotkeys()
    sys.exit()

In [ ]:
f= open("guru99.txt","w+")